In [1]:
import time
import tempfile
import json

from tqdm import tqdm
import pandas as pd
import ray
import click
import torch

from srtml import serve
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import seaborn as sns
sns.set(style='whitegrid', palette='muted')


In [2]:
serve.init()

2020-12-07 13:50:34,116	INFO resource_spec.py:212 -- Starting Ray with 324.56 GiB memory available for workers and up to 0.09 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-12-07 13:50:34,500	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [1]:
from transformers import BertTokenizer, BertConfig
import torch
from transformers.modeling_bert import BertLayer, BertEmbeddings, BertPooler, BertPreTrainingHeads
from torch.nn import LayerNorm as BertLayerNorm

class Bert(torch.nn.Module):
    def __init__(self, config):
        super(Bert, self).__init__()
        self.embedding_layer = BertEmbeddings(config)
        self.layers = []
        for i in range(config.num_hidden_layers-12):
            self.layers.append(BertLayer(config))

        self.layers = torch.nn.ModuleList(self.layers)
        self.pooling_layer = BertPooler(config)
        self.pre_training_heads_layer = BertPreTrainingHeads(config)
        self.config = config;
        self.apply(self.init_bert_weights)

    def init_bert_weights(self, module):
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding)):
            module.weight.data.normal_(mean=0.0,
                                       std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, torch.nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, input0, input1):
        out0 = input0
        out1 = input1
        out = self.embedding_layer(out0, out1)
        for layer in self.layers:
            out,  = layer(out)
        # out2 = self.pooling_layer(out)
        # out3 = self.pre_training_heads_layer(out, out2)
        return out


class Stage0(torch.nn.Module):
    def __init__(self, config):
        super(Stage0, self).__init__()
        self.embedding_layer = BertEmbeddings(config)
        self.layers = []
#         print(config.num_hidden_layers)
        for i in range(config.num_hidden_layers // 12):
            self.layers.append(BertLayer(config))
        len(self.layers)
        self.layers = torch.nn.ModuleList(self.layers)
        self.config = config
        self.apply(self.init_bert_weights)

    def init_bert_weights(self, module):
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding)):
            module.weight.data.normal_(mean=0.0,
                                       std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, torch.nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, input0, input1):
        out0 = input0
        out1 = input1
        out = self.embedding_layer(out0, out1)
        for layer in self.layers:
            out,  = layer(out)
        return out

class Stage1(torch.nn.Module):
    def __init__(self, config):
        super(Stage1, self).__init__()
        self.layers = []
        for i in range(config.num_hidden_layers // 12):
            self.layers.append(BertLayer(config))
        len(self.layers)
        self.layers = torch.nn.ModuleList(self.layers)
#         self.pooling_layer = BertPooler(config)
#         self.pre_training_heads_layer = BertPreTrainingHeads(config)
        self.config = config
        self.apply(self.init_bert_weights)

    def init_bert_weights(self, module):
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding)):
            module.weight.data.normal_(mean=0.0,
                                       std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, torch.nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, input0):
        out = input0
        for layer in self.layers:
            out,  = layer(out)
#         out2 = self.pooling_layer(out)
#         out3 = self.pre_training_heads_layer(out, out2)
        return out

class Stage2(torch.nn.Module):
    def __init__(self, config):
        super(Stage2, self).__init__()
        self.layers = []
        for i in range(config.num_hidden_layers // 12):
            self.layers.append(BertLayer(config))
        len(self.layers)
        self.layers = torch.nn.ModuleList(self.layers)
#         self.pooling_layer = BertPooler(config)
#         self.pre_training_heads_layer = BertPreTrainingHeads(config)
        self.config = config
        self.apply(self.init_bert_weights)

    def init_bert_weights(self, module):
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding)):
            module.weight.data.normal_(mean=0.0,
                                       std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, torch.nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, input0):
        out = input0
        for layer in self.layers:
            out,  = layer(out)
#         out2 = self.pooling_layer(out)
#         out3 = self.pre_training_heads_layer(out, out2)
        return out

class Stage3(torch.nn.Module):
    def __init__(self, config):
        super(Stage3, self).__init__()
        self.layers = []
        for i in range(config.num_hidden_layers // 12):
            self.layers.append(BertLayer(config))
        len(self.layers)
        self.layers = torch.nn.ModuleList(self.layers)
        self.pooling_layer = BertPooler(config)
        self.pre_training_heads_layer = BertPreTrainingHeads(config)
        self.config = config
        self.apply(self.init_bert_weights)

    def init_bert_weights(self, module):
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding)):
            module.weight.data.normal_(mean=0.0,
                                       std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        if isinstance(module, torch.nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, input0):
        out = input0
        for layer in self.layers:
            out,  = layer(out)
#         out2 = self.pooling_layer(out)
#         out3 = self.pre_training_heads_layer(out, out2)
        return out

##################################################################################################


bert_config = BertConfig.from_pretrained('bert-large-uncased')

In [3]:
txt = 'Debugging'
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
encoded = tokenizer(text=txt, add_special_tokens=True,  # Add [CLS] and [SEP]
                         max_length = 64,  # maximum length of a sentence
                         padding='max_length',  # Add [PAD]s
                         return_attention_mask = True,  # Generate the attention mask
                         return_tensors = 'pt')
inp = torch.rand(64, 1024)

In [4]:
bs = 8
gpu = 4
inp1 = torch.stack([encoded['input_ids'][0]]*bs)
inp2 = torch.stack([encoded['attention_mask'][0]]*bs)
inp3 = torch.stack([inp]*bs)
inp3.shape

torch.Size([8, 64, 1024])

In [5]:
for bs, gpu in zip([1,2,4,8], [0,1,2,3]):
    inp1 = torch.stack([encoded['input_ids'][0]]*bs)
    inp2 = torch.stack([encoded['attention_mask'][0]]*bs)
    inp3 = torch.stack([inp]*bs)

    se = Stage0(bert_config).cuda(gpu)
    se2 = Stage3(bert_config).cuda(gpu+4)
    se(inp1.cuda(gpu), inp2.cuda(gpu))
    se2(inp3.cuda(gpu+4))

In [4]:
return_type = 'torch'
batch_size = 1
def noop(*args, **kwargs):
    bs = serve.context.batch_size
    time.sleep(0.1)
#     assert (
#         bs == batch_size
#     ), f"worker received {batch_size} which is not what expected {batch_size}"
    result = ""
    
    if return_type == "torch":
        result = torch.zeros((3, 224, 224))

    if bs is None:  # No batching
        return result
    else:
        return [result] * bs

noop = serve.accept_batch(noop)

In [5]:
bert_config = BertConfig.from_pretrained('bert-large-uncased')

class BertClass:
    def __init__(self, model):
        self.model = model
        self.model = self.model.cuda()
    
    def __call__(self, data: list) -> list:
        input0 = torch.stack([item['input_ids'][0] for item in data])
        input1 = torch.stack([item['attention_mask'][0] for item in data])

        
        input0 = input0.cuda()
        input1 = input1.cuda()
        outputs = self.model(input0, input1)
        res = list(outputs.cpu().unbind())
        return res

def bert(*args, data, **kwargs):

    model = Bert(bert_config)
    model.cuda()
    input0 = torch.stack([item['input_ids'][0] for item in data])
    input1 = torch.stack([item['attention_mask'][0] for item in data])

    input0 = input0.cuda()
    input1 = input1.cuda()
    outputs = model(input0, input1)
    res = list(outputs.cpu().unbind())
    return res


def stage0(*args, data, **kwargs):

    model = Stage0(bert_config)
#     model.cuda()
    input0 = torch.stack([item['input_ids'][0] for item in data])
    input1 = torch.stack([item['attention_mask'][0] for item in data])

    input0 = input0.cuda()
    input1 = input1.cuda()
    outputs = model(input0, input1)
    res = list(outputs.cpu().unbind())
    return res

def stage1(*args, data, **kwargs):
    model = Stage1(bert_config)
    model.cuda()
#     print('stage1: ', data[0].shape)
    input0 = torch.stack(data)
    input0 = input0.cuda()
    outputs = model(input0)
    res = list(outputs.cpu().unbind())
    return res

def stage2(*args, data, **kwargs):
    model = Stage2(bert_config)
    model.cuda()
#     print('stage2: ', data[0].shape)
    input0 = torch.stack(data)
    input0 = input0.cuda()
    outputs = model(input0)
    res = list(outputs.cpu().unbind())
    return res

def stage3(*args, data, **kwargs):
    model = Stage3(bert_config)
    model.cuda()
#     print('stage3: ', data[0].shape)
    input0 = torch.stack(data)
    input0 = input0.cuda()
    outputs = model(input0)
    res = list(outputs.cpu().unbind())
    return res

Bert = serve.accept_batch(Bert)
bert = serve.accept_batch(bert)
stage3 = serve.accept_batch(stage3)
stage2 = serve.accept_batch(stage2)
stage1 = serve.accept_batch(stage1)
stage0 = serve.accept_batch(stage0)

In [6]:

with serve.using_router("noop"):
    serve.create_endpoint("noop", "/noop")
    config = serve.BackendConfig(max_batch_size=batch_size)
#     config = serve.BackendConfig()
    serve.create_backend(noop, "noop", backend_config=config)
    serve.link("noop", "noop")
    handle = serve.get_handle("noop")

num_backends = 1
backend = [Bert]#[stage0, stage1, stage2, stage3]
handle = [None]*num_backends

for i in range(num_backends):
    router_name = 'stage' + str(i)
    with serve.using_router(router_name):
        serve.create_endpoint(router_name, '/'+router_name)
        config = serve.BackendConfig(max_batch_size=batch_size, num_gpus=1)#, resources={'Tesla P40':1})
    #     config = serve.BackendConfig()
        serve.create_backend(backend[i], router_name, backend_config=config)
        serve.link(router_name, router_name)
        handle[i] = serve.get_handle(router_name)

# with serve.using_router("noop2"):
#     serve.create_endpoint("noop2", "/noop2")
#     config = serve.BackendConfig(max_batch_size=batch_size, num_gpus=1)
# #     config = serve.BackendConfig()
#     serve.create_backend(stage1, "noop2", backend_config=config)
#     serve.link("noop2", "noop2")
#     handle2 = serve.get_handle("noop2")

num_queries = 30
num_warmups = 20

In [33]:
latency = []
txt = 'Debugging'
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
encoded = tokenizer(text=txt, add_special_tokens=True,  # Add [CLS] and [SEP]
                         max_length = 64,  # maximum length of a sentence
                         padding='max_length',  # Add [PAD]s
                         return_attention_mask = True,  # Generate the attention mask
                         return_tensors = 'pt')
print(encoded['input_ids'].shape)
prepoc_dummy_kwarg = {"data": [txt]}

latency_list_ms = []
for _ in range(10):
    start_time = time.time()
    # x = ray.wait([ppu_handle.enqueue_batch(data=[encoded])], num_returns=1)
    x = ray.get([handle[0].enqueue_batch(data=[encoded])])
    print(x)
    end_time = time.time()
    time_taken_ms = (end_time - start_time) * 1000
    latency_list_ms.append(time_taken_ms)
pprint(latency_list_ms)

for i in tqdm(range(num_warmups + num_queries)):
    if i == num_warmups:
        serve.clear_trace()
    oids = []
    start = time.perf_counter()
    x = None
    if not batch_size:
        ray.get(
            # This is how to pass a higher level metadata to the tracing
            # context
            handle.options(
                tracing_metadata={"pipeline-id": i}
            ).remote()
        )
    else:
#         r = handle1.options(tracing_metadata={"pipeline-id": i}).remote(
#                 data=encoded
#             )
#         r = handle2.options(tracing_metadata={"pipeline-id": i}).remote(
#                  data=r  # torch tensor
#             )
#         oids.append(r)

#         x = ray.get(handle1.enqueue_batch(val=ray.get(handle.enqueue_batch(val=[1] * batch_size))))
        #x = ray.get(handle2.options(tracing_metadata={"pipeline-id": i}).remote(data=ray.get(handle1.options(tracing_metadata={"pipeline-id": i}).remote(data = encoded))))
#         ray.get([handle.remote() for _ in range(batch_size)])
#         print(x)
        x = ray.get([handle[0].enqueue_batch(data=[encoded])])
#         x1 = ray.get(handle[0].options(tracing_metadata={'pipeline-id':i}).remote(data=encoded))
#         x2 = ray.get(handle[1].options(tracing_metadata={'pipeline-id':i}).remote(data=ray.get(handle[0].options(tracing_metadata={'pipeline-id':i}).remote(data=encoded))))
#         x3 = ray.get(handle[2].options(tracing_metadata={'pipeline-id':i}).remote(data=ray.get(handle[1].options(tracing_metadata={'pipeline-id':i}).remote(data=ray.get(handle[0].options(tracing_metadata={'pipeline-id':i}).remote(data=encoded))))))
#         x4 = ray.get(handle[1].options(tracing_metadata={"pipeline-id": i}).remote(data=ray.get(handle[0].options(tracing_metadata={"pipeline-id": i}).remote(data = encoded))))
#         x4 = ray.get(handle[3].options(tracing_metadata={'pipeline-id':i}).remote(data=ray.get(handle[2].options(tracing_metadata={'pipeline-id':i}).remote(data=ray.get(handle[1].options(tracing_metadata={'pipeline-id':i}).remote(data=ray.get(handle[0].options(tracing_metadata={'pipeline-id':i}).remote(data=encoded))))))))
#     x = ray.wait(oids, len(oids))
    end = time.perf_counter()
    latency.append(end - start)
    
# Remove initial samples
latency = latency[num_warmups:]

torch.Size([1, 64])
[[tensor([[ 0.8785, -0.6656, -0.9790,  ..., -0.3431,  0.5983,  0.7632],
        [ 0.0077,  0.0257, -2.0476,  ..., -0.6537,  0.9058,  0.6314],
        [ 0.9455, -0.5244, -1.6135,  ...,  0.4624,  0.7413,  0.0031],
        ...,
        [ 0.8474, -0.1775, -1.9153,  ..., -0.8308, -0.5692,  0.7796],
        [ 0.1397, -0.7879, -1.0723,  ..., -1.4896,  0.3921,  0.1265],
        [ 0.9242, -0.0896, -0.6089,  ..., -0.8795, -0.9089,  0.3433]],
       requires_grad=True)]]
[[tensor([[ 2.2249, -0.2349,  0.0603,  ..., -0.5012, -0.2063,  0.8447],
        [ 1.1882, -0.8061, -0.9110,  ...,  0.1674, -0.9667,  1.0657],
        [ 1.1026,  0.1346, -0.4158,  ...,  0.2178, -0.2201,  1.0791],
        ...,
        [ 1.4115, -1.6898,  0.4669,  ..., -0.3013, -1.0541, -0.0807],
        [ 1.6818, -0.6471, -0.0762,  ...,  0.1737,  0.1678, -0.2517],
        [ 2.2949, -1.0060, -0.6091,  ..., -0.5323, -0.9178,  0.8164]],
       requires_grad=True)]]
[[tensor([[ 0.2354, -0.4642,  0.0224,  ..., -0.675

KeyboardInterrupt: 

[6479.653358459473,
 2719.071388244629,
 2703.2644748687744,
 2689.974069595337,
 2713.3231163024902,
 2717.5133228302,
 2701.0676860809326,
 2712.5818729400635,
 2882.8675746917725,
 2726.3646125793457]

In [ ]:
latency_list_ms = []
for _ in range(10):
    start_time = time.time()
    ray.wait([handle[0].enqueue_batch(data=[encoded])], num_returns=1)
    end_time = time.time()
    time_taken_ms = (end_time - start_time) * 1000
    latency_list_ms.append(time_taken_ms)


In [ ]:
latency_list_ms

In [ ]:

series = pd.Series(latency) * 1000
print("Latency for single noop backend (ms)")
print(series.describe(percentiles=[0.5, 0.9, 0.95, 0.99]))

In [ ]:
trace_file = 'trace.json'
with open(trace_file, "w") as f:
    json.dump(serve.get_trace(), f)
print(f"trace file written to {trace_file}")

In [ ]:
data = serve.get_trace()
df = pd.DataFrame(data['metadata'])
df = df.transpose()
print(df)
df = pd.DataFrame(data['traces'])
df = df.sort_values(['query_id', 'event', 'time_us']).reset_index(drop=True)

In [ ]:
df = pd.DataFrame(data['traces'])
df = df.sort_values(['query_id', 'event', 'time_us']).reset_index(drop=True)
df1 = pd.DataFrame()
c = 0
prev = None
for a,q,i in df.itertuples(index=False):
#     print(q)
    if i == prev:
        c+=1
        df1 = df1.append({'query_id': str(q), 'event': i+'_'+str(c)}, ignore_index = True)
    else:
        c = 0
        df1 = df1.append({'query_id': str(q), 'event': i+'_'+str(c)}, ignore_index = True)
        prev = i
# df.loc[df['query_id'] == 514]
# df.head(20)
df1.head(20)
df[['event', 'query_id']] = df1
df.head(50)
# df = df.sort_values('time_us')
# df.head(20)


In [ ]:
df = df.pivot(index='query_id', columns="event").droplevel(0,axis=1)

In [ ]:
event_ordering = [
    'router_enqueue',
    "router_dequeue",
    "worker_start",
    "worker_done",
    "router_recv_result"
]
new_event_ordering = []
for i in range(num_backends):
    for event in event_ordering:
          new_event_ordering.append(event+ '_' + str(i)) 
event_ordering = new_event_ordering
series = {}
for e_start, e_end in zip(event_ordering, event_ordering[1:]):
    series[f"{e_start}->{e_end}"] = (df[e_end] - df[e_start])/1000
diff_df = pd.DataFrame(series)

diff_df.to_csv('data.csv')
print(diff_df.mean(axis=0))
# diff_df

In [ ]:
for name, s in diff_df.T.iterrows():
    sns.distplot(s, label=name, kde_kws={"cumulative": True})
plt.legend()
plt.xlabel("us")